In [1]:
import os
import json
import psycopg2
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
import folium
import folium.plugins
import fiona
from pyproj import Proj, transform
import osmnx as ox
import networkx as nx
import matplotlib.colors as colors
import matplotlib.cm as cm
from shapely.ops import cascaded_union
import geojson
import operator

In [2]:
nx.__version__

'2.1'

In [3]:
import matplotlib.cm as cmx
import matplotlib.colors as colors

def style_function(feature):
    color = feature['properties']['color']
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': "rgb(0, 0, 255)" if color == 2 else "rgb(255, 0, 0)" 
    }

In [4]:
ox.config(log_file=True, log_console=False, use_cache=True)

In [5]:
location = [47.22662, 8.81834]

In [6]:
tags = {
  "landuse_tags": [
    "retail"
  ],
  "amenity_tags": [
    "pub",
    "bar",
    "cafe",
    "restaurant",
    "pharmacy",
    "bank",
    "fast_food",
    "food_court",
    "ice_cream",
    "library",
    "ferry_terminal",
    "clinic",
    "doctors",
    "hospital",
    "pharmacy",
    "veterinary",
    "dentist",
    "arts_centre",
    "cinema",
    "community_centre",
    "casino",
    "fountain",
    "nightclub",
    "studio",
    "theatre",
    "dojo",
    "internet_cafe",
    "marketplace",
    "post_opffice",
    "townhall"
  ],
  "shop_tags": [
    "mall",
    "bakery",
    "beverages",
    "butcher",
    "chocolate",
    "coffee",
    "confectionery",
    "deli",
    "frozen_food",
    "greengrocer",
    "healthfood",
    "ice_cream",
    "pasta",
    "pastry",
    "seafood",
    "spices",
    "tea",
    "department_store",
    "supermarket",
    "bag",
    "boutique",
    "clothes",
    "fashion",
    "jewelry",
    "leather",
    "shoes",
    "tailor",
    "watches",
    "chemist",
    "cosmetics",
    "hairdresser",
    "medical_supply",
    "electrical",
    "hareware",
    "electronics",
    "sports",
    "swimming_pool",
    "collector",
    "games",
    "music",
    "books",
    "gift",
    "stationery",
    "ticket",
    "laundry",
    "pet",
    "tobacco",
    "toys"
  ],
  "leisure_tags": [
    "adult_gaming_centre",
    "amusement_arcade",
    "beach_resort",
    "fitness_centre",
    "garden",
    "ice_rink",
    "sports_centre",
    "water_park"
  ]
}


In [15]:
class AoiQueryGenerator():
    def __init__(self, location=None, hull_algorithm='convex'):
        self.location = location
        self.hull_algorithm = hull_algorithm

        ox.utils.config(cache_folder='tmp/cache', use_cache=True)

    def bbox_query(self):
        location_3857 = transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), self.location[1], self.location[0])
        location_3857 = " ".join([str(coordinate) for coordinate in location_3857])

        return """
    (SELECT ST_Buffer(ST_GeomFromText('POINT({})', 3857), 1000) AS bbox)
        """.format(location_3857)

    def polygons_query(self):
        if self.location is None:
            return "SELECT * FROM pois"

        return """
        SELECT * FROM pois WHERE st_intersects(geometry, {bbox})
        """.format(bbox=self.bbox_query())

    def preclusters_subset_query(self):
        if self.location is None:
            return "SELECT * FROM preclusters"

        return """
        SELECT * FROM preclusters WHERE st_intersects(hull, {bbox})
        """.format(bbox=self.bbox_query())

    def clusters_query(self):
        return """
        WITH preclusters_subset AS ({preclusters_subset_query})
        SELECT preclusters_subset.id AS precluster_id,
               geometry,
               ST_ClusterDBSCAN(geometry, eps := 35, minpoints := preclusters_subset.dbscan_minpts) over () AS cid
        FROM pois, preclusters_subset
        WHERE ST_Within(geometry, preclusters_subset.hull)
        """.format(preclusters_subset_query=self.preclusters_subset_query())

    def hulls_query(self):
        if self.hull_algorithm == 'concave':
            hull = 'ST_ConcaveHull(ST_Union(geometry), 0.999)'
        else:
            hull = "ST_ConvexHull(ST_Union(geometry))"

        return """
        WITH clusters AS ({clusters_query})
        SELECT cid, {hull} AS geometry
        FROM clusters
        WHERE cid IS NOT NULL
        GROUP BY cid
        """.format(clusters_query=self.clusters_query(), hull=hull)

    def clusters_and_hulls_query(self):
        return """
        WITH clusters AS ({clusters_query}),
        hulls AS ({hulls_query})
        SELECT cid, geometry FROM clusters
        UNION ALL
        SELECT cid, geometry FROM hulls
            """.format(clusters_query=self.clusters_query(), hulls_query=self.hulls_query())

    def network_centrality_query(self):
        return self.extended_hulls_query() + """

UNION

SELECT 2 as color, geometry FROM hulls

UNION

SELECT 3 as color, geometry FROM intersecting_lines
"""

    def extended_hulls_query(self):
        aois = self.query_database(self.hulls_query())
        aois = aois.to_crs(fiona.crs.from_epsg(4326))
        central_nodes = []

        for aoi in tqdm(aois.geometry):
            try:
                aoi_graph = ox.graph_from_polygon(aoi.buffer(0.001), network_type='all')
                closeness_centrality = nx.closeness_centrality(aoi_graph)
                sorted_nodes = sorted(closeness_centrality.items(), key=operator.itemgetter(1), reverse=True)
                central_nodes += [node[0] for node in sorted_nodes[:len(sorted_nodes) // 10]]
            except KeyboardInterrupt:
                # leave on ctrl-c
                sys.exit(0)
            except:
                print("fetching graph failed for {}".format(aoi))

        central_nodes_ids = ', '.join([f'{key}' for key in central_nodes])

        return """
WITH hulls AS ({hulls_query}),
intersecting_lines AS (
    SELECT hulls.cid, ST_Intersection(way, ST_Buffer(hulls.geometry, 50)) AS geometry FROM planet_osm_line, hulls
    WHERE osm_id = ANY(
      SELECT id FROM planet_osm_ways
      WHERE nodes && ARRAY[{central_nodes_ids}]::bigint[]
    )
    AND ST_DWithin(planet_osm_line.way, hulls.geometry, 50)
)

SELECT 1 AS color, ST_ConcaveHull(ST_Union(geometry), 0.99) AS geometry FROM (
  SELECT cid, geometry FROM hulls
  UNION
  SELECT cid, geometry FROM intersecting_lines
) AS tmp
GROUP BY cid
""".format(hulls_query=self.hulls_query(), central_nodes_ids=central_nodes_ids)

    def without_water_query(self, hulls_query):
        return """WITH hulls AS ({hulls_query})
SELECT ST_Difference(hulls.geometry, coalesce((
    SELECT ST_Union(way) AS geometry FROM planet_osm_polygon
    WHERE (water IS NOT NULL OR waterway IS NOT NULL)
          AND (tunnel IS NULL OR tunnel = 'no')
    AND st_intersects(way, hulls.geometry)
), 'GEOMETRYCOLLECTION EMPTY'::geometry)) AS geometry
FROM hulls""".format(hulls_query=hulls_query)

    def cascade_aois_query(self, aois_query):
        return """
WITH aois AS ({aois_query})
SELECT (ST_Dump(ST_Union(geometry))).geom AS geometry FROM aois
""".format(aois_query=aois_query)



In [17]:
eps = 35

hulls_query = AoiQueryGenerator(location=location).hulls_query()

In [18]:
with psycopg2.connect("") as conn:
    aois = gpd.read_postgis(hulls_query, conn, geom_col='geometry')
    
    aois.crs = fiona.crs.from_epsg(3857)
    
    aois_4326 = aois.to_crs(fiona.crs.from_epsg(4326))


In [29]:
m = folium.Map(location=location, zoom_start=16)
folium.plugins.Fullscreen().add_to(m)

folium.GeoJson(aois).add_to(m)

m

In [20]:
for aoi in aois_4326.geometry:
    aoi_graph = ox.graph_from_polygon(aoi.buffer(0.001), network_type='all')
    ox.plot_graph_folium(aoi_graph, graph_map=m, popup_attribute='name', edge_width=2)
 
m.location = location
m.zoom_start = 16
m

In [21]:
aois_4326 = aois.to_crs(fiona.crs.from_epsg(4326))

all_nodes_centrality = {}
top_20_percent_nodes_centrality = []
for aoi in aois_4326.geometry:
    aoi_graph = ox.graph_from_polygon(aoi.buffer(0.001), network_type='all')
    closeness_centrality = nx.closeness_centrality(aoi_graph)
    all_nodes_centrality = {**all_nodes_centrality, **closeness_centrality}
    
    sorted_nodes = sorted(closeness_centrality.items(), key=operator.itemgetter(1), reverse=True)
    top_20_percent_nodes_centrality += [node[0] for node in sorted_nodes[:len(sorted_nodes) // 10]]

In [22]:
points_query = """
SELECT st_transform(st_geomfromtext('POINT ('||lon/100||' '||lat/100||')',900913), 3857) AS geometry
FROM planet_osm_nodes
WHERE id = ANY(ARRAY[{}])
"""

all_node_ids = ', '.join([f'{key}' for key in all_nodes_centrality.keys()])

query = points_query.format(all_node_ids)
central_points = gpd.read_postgis(query, conn, geom_col='geometry')
central_points.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=location, zoom_start=16)
folium.plugins.Fullscreen().add_to(m)
#print(nodes_centrality.keys() - list(central_points.osm_id))

if(central_points.size > 0): {
    folium.GeoJson(central_points).add_to(m)
}
        
m

In [23]:
joined_node_ids = ', '.join([f'{key}' for key in top_20_percent_nodes_centrality])

In [24]:
points_query = """
SELECT st_transform(st_geomfromtext('POINT ('||lon/100||' '||lat/100||')',900913), 3857) AS geometry
FROM planet_osm_nodes
WHERE id = ANY(ARRAY[{}])
"""

query = points_query.format(joined_node_ids)
central_points = gpd.read_postgis(query, conn, geom_col='geometry')
central_points.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=location, zoom_start=16)
folium.plugins.Fullscreen().add_to(m)

if(central_points.size > 0): {
    folium.GeoJson(central_points).add_to(m)
}
        
m
    

In [27]:
ways_query = """
WITH hulls AS ({})
SELECT way AS geometry FROM planet_osm_line, hulls 
WHERE osm_id = ANY(
  SELECT id FROM planet_osm_ways
  WHERE nodes && ARRAY[{}]::bigint[]
)
AND ST_DWithin(hulls.geometry, planet_osm_line.way, 50) 
"""

m = folium.Map(location=location, zoom_start=16)
folium.plugins.Fullscreen().add_to(m)

with psycopg2.connect("") as conn:
    query = ways_query.format(hulls_query, joined_node_ids)
    central_points = gpd.read_postgis(query, conn, geom_col='geometry')
    central_points.crs = fiona.crs.from_epsg(3857)

    if(central_points.size > 0): {
        folium.GeoJson(central_points).add_to(m)
    }

m

In [30]:
ways_query = """
WITH hulls AS ({})
SELECT ST_Intersection(way, ST_Buffer(hulls.geometry, 50)) AS geometry FROM planet_osm_line, hulls 
WHERE osm_id = ANY(
  SELECT id FROM planet_osm_ways
  WHERE nodes && ARRAY[{}]::bigint[]
)
AND ST_DWithin(hulls.geometry, planet_osm_line.way, 50) 
"""

#m = folium.Map(location=location, zoom_start=16)
#folium.plugins.Fullscreen().add_to(m)

with psycopg2.connect("") as conn:
    query = ways_query.format(hulls_query, joined_node_ids)
    central_points = gpd.read_postgis(query, conn, geom_col='geometry')
    central_points.crs = fiona.crs.from_epsg(3857)

    if(central_points.size > 0): {
        folium.GeoJson(central_points).add_to(m)
    }

m

In [31]:
ways_query = """
WITH hulls AS ({}),
intersecting_lines AS (
    SELECT hulls.cid, ST_Intersection(way, ST_Buffer(hulls.geometry, 50)) AS geometry FROM planet_osm_line, hulls 
    WHERE osm_id = ANY(
      SELECT id FROM planet_osm_ways
      WHERE nodes && ARRAY[{}]::bigint[]
    )
    AND ST_DWithin(planet_osm_line.way, hulls.geometry, 50)
)

SELECT 1 AS color, ST_ConcaveHull(ST_Union(geometry), 0.99) AS geometry FROM (
  SELECT cid, geometry FROM hulls
  UNION
  SELECT cid, geometry FROM intersecting_lines
) AS tmp
GROUP BY cid

UNION

SELECT 2 as color, geometry FROM hulls
"""


with psycopg2.connect("") as conn:
    query = ways_query.format(hulls_query, joined_node_ids)
    central_points = gpd.read_postgis(query, conn, geom_col='geometry')
    central_points.crs = fiona.crs.from_epsg(3857)

In [33]:
m = folium.Map(location=location, zoom_start=16)
folium.plugins.Fullscreen().add_to(m)

if(central_points.size > 0): {
    folium.GeoJson(central_points, style_function=style_function).add_to(m)
}

m